This code automates a web browser to play a cookie-clicker-like game. 

In [ ]:
from selenium import webdriver
import time

def initialize_driver():
    chrome_driver_path = YOUR_CHROME_DRIVER_PATH
    driver = webdriver.Chrome(chrome_driver_path)
    driver.get("http://orteil.dashnet.org/experiments/cookie/")
    return driver

def click_cookie(driver):
    cookie = driver.find_element_by_id("cookie")
    cookie.click()

def get_upgrade_items(driver):
    items = driver.find_elements_by_css_selector("#store div")
    item_ids = [item.get_attribute("id") for item in items]
    return item_ids

def get_upgrade_prices(driver):
    all_prices = driver.find_elements_by_css_selector("#store b")
    item_prices = []
    for price in all_prices:
        element_text = price.text
        if element_text != "":
            cost = int(element_text.split("-")[1].strip().replace(",", ""))
            item_prices.append(cost)
    return item_prices

def get_cookie_count(driver):
    money_element = driver.find_element_by_id("money").text
    if "," in money_element:
        money_element = money_element.replace(",", "")
    cookie_count = int(money_element)
    return cookie_count

def get_affordable_upgrades(cookie_count, item_prices, item_ids):
    cookie_upgrades = {}
    for n in range(len(item_prices)):
        cookie_upgrades[item_prices[n]] = item_ids[n]
    affordable_upgrades = {}
    for cost, id in cookie_upgrades.items():
        if cookie_count > cost:
            affordable_upgrades[cost] = id
    return affordable_upgrades

def purchase_upgrade(driver, to_purchase_id):
    driver.find_element_by_id(to_purchase_id).click()

def main():
    driver = initialize_driver()
    item_ids = get_upgrade_items(driver)

    timeout = time.time() + 5
    five_min = time.time() + 60*5

    while True:
        click_cookie(driver)

        if time.time() > timeout:
            item_prices = get_upgrade_prices(driver)
            cookie_count = get_cookie_count(driver)
            affordable_upgrades = get_affordable_upgrades(cookie_count, item_prices, item_ids)

            if affordable_upgrades:
                highest_price_affordable_upgrade = max(affordable_upgrades)
                to_purchase_id = affordable_upgrades[highest_price_affordable_upgrade]
                purchase_upgrade(driver, to_purchase_id)

            timeout = time.time() + 5

        if time.time() > five_min:
            cookie_per_s = driver.find_element_by_id("cps").text
            print(cookie_per_s)
            break

    driver.quit()

if __name__ == "__main__":
    main()
